In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
from tqdm import tqdm

from torch.utils import data
from transformers import AdamW, get_linear_schedule_with_warmup, AutoModelForMaskedLM, AutoConfig, AutoTokenizer, DataCollatorWithPadding

In [20]:
model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# add [HL], [/HL] token
added_token_num = tokenizer.add_special_tokens({"additional_special_tokens": ["[HL]", "[/HL]"]})
# add token number
model.resize_token_embeddings(tokenizer.vocab_size + added_token_num)

model = nn.DataParallel(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
model_path = "model_weights_4_72.512201225647.pth"
model.load_state_dict(torch.load(model_path))
model.to('cuda')

DataParallel(
  (module): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30524, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm

In [22]:

file_path = "./data/squad_nqg/test.json"
with open(file_path, 'r') as file:
    data = json.load(file)

In [23]:
data[0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}]}

In [68]:
context_list = []
question_list = []

cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
mask_token = tokenizer.mask_token

for d in tqdm(data[:5]):
    answers = d['answers'][0]['text']
    answer_start = d["answers"][0]["answer_start"]
    question_list.append(d['question'][:-1]) #?제외
    input_text = f"{cls_token} {d['context'][:answer_start]} [HL] {d['context'][answer_start:answer_start+len(answers)]} [/HL] {d['context'][answer_start+len(answers):]} {sep_token}"
    context_list.append(input_text)

100%|██████████| 5/5 [00:00<00:00, 40329.85it/s]


In [70]:
model.eval()
pred_sentence_list = [] # 예측된 문장이 저장됨
for i, context in enumerate(tqdm(context_list)):
    context_tokenized = tokenizer.encode(context, add_special_tokens = False) # token id --> [CLS] + passage with [HL] + [SEP]
    pred_str_list = [] # 새롭게 예측된 토큰이 저장됨
    
    for k in range(20): # 최대 토큰 개수 20개로 제한
        pred_str_ids = tokenizer.convert_tokens_to_ids(pred_str_list + [mask_token])
        predict_token = context_tokenized + pred_str_ids
        # print(tokenizer.decode(predict_token))
        predict_token = torch.tensor([predict_token])
        predictions = model(predict_token) # k: [CLS] + tokenized passage with [HL] + [SEP] + predicted question + [MASK]
        predicted_index = torch.argmax(predictions[0][0][-1]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
        if '[SEP]' in predicted_token:
            break
        pred_str_list.append(predicted_token[0])
    token_ids = tokenizer.convert_tokens_to_ids(pred_str_list)
    
    pred_sentence_list.append(tokenizer.decode(token_ids))

100%|██████████| 5/5 [00:01<00:00,  3.71it/s]


In [71]:
pred_sentence_list

['who was reputed to have appeared in the museum of mary in 1858?',
 "what is the main building's gold dome?",
 'what is the basilica of the sacred heart?',
 'what is the grotto?',
 "what is atop the main building's gold dome?"]

In [ ]:
with open('./test.txt','w',encoding='UTF-8') as f:
    for name in pred_sentence_list:
        f.write(name+'\n')